# **Import**

In [1]:
# Cell 1: Setup imports
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Thêm đường dẫn gốc của dự án vào sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import TradingEngine và components
from src.core.trading_engine import TradingEngine
from src.strategies.sma_crossover import SMACrossoverStrategy
from src.strategies.rsi_strategy import RSIStrategy
from src.strategies.macd_strategy import MACDStrategy
from src.utils.config_manager import ConfigManager
from src.data.data_manager import DataManager

# Import visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

print("Import thành công!")

Import thành công!


# **Data**

In [2]:
intervals = ['daily', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'min_1', 
'min_15', 'min_3', 'min_30', 'min_45', 'min_5', 'monthly', 'weekly']

In [3]:
# Cell 2: Khởi tạo TradingEngine
config_path = "../../config/config.yaml"
engine = TradingEngine(config_path)

print(f"TradingEngine đã được khởi tạo với vốn ban đầu: ${engine.cash:,.2f}")
print(f"Config loaded: {engine.config.get('trading.symbols')}")
print(f"Config loaded: {engine.config.get('data.source')}")

# Cell 3: Chuẩn bị data
# Sử dụng data có sẵn hoặc lấy từ DataManager
symbols = engine.config.get("trading.symbols", ["AAPL"])
start_date = engine.config.get("data.start_date", "2023-01-01")
end_date = engine.config.get("data.end_date", "2023-12-31")


# Lấy data từ DataManager
data_manager = DataManager(engine.config)
historical_data = data_manager.get_historical_data(
    symbols=symbols,
    start_date=start_date,
    end_date=end_date,
    interval=engine.config.get("data.interval", "1d"),
    n_bars=engine.config.get("data.n_bars", 1000)
)

print(f"Lấy data cho symbols: {symbols}")
# print(f"Period: {start_date} đến {end_date}")
print(f"Data shape: {historical_data.shape}")
print(f"Data columns: {historical_data.columns.tolist()}")
print(
    f"Date range: {historical_data.index.min()} \
        đến {historical_data.index.max()} \
            interval: {engine.config.get('data.interval', '1d')}"
)

2025-08-02 14:53:21.089 | INFO     | src.utils.config_manager:__init__:31 - Configuration loaded from ../../config/config.yaml
2025-08-02 14:53:21.090 | INFO     | src.risk.risk_manager:__init__:49 - Risk manager initialized
2025-08-02 14:53:21.091 | INFO     | src.core.trading_engine:__init__:77 - Trading engine initialized with $100,000.00 initial capital
2025-08-02 14:53:21.092 | INFO     | src.data.data_manager:get_historical_data:64 - Fetching historical OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2025-07-31


TradingEngine đã được khởi tạo với vốn ban đầu: $100,000.00
Config loaded: ['Bitstamp:BTCUSD']
Config loaded: tradingview


2025-08-02 14:53:21.755 | INFO     | src.data.data_manager:_fetch_tradingview_ohlcv_data:159 - Successfully fetched OHLCV data for 1 symbols
2025-08-02 14:53:21.759 | INFO     | src.data.data_manager:get_historical_data:97 - OHLCV data cached successfully


Lấy data cho symbols: ['Bitstamp:BTCUSD']
Data shape: (100, 5)
Data columns: [('Bitstamp:BTCUSD', 'open'), ('Bitstamp:BTCUSD', 'high'), ('Bitstamp:BTCUSD', 'low'), ('Bitstamp:BTCUSD', 'close'), ('Bitstamp:BTCUSD', 'volume')]
Date range: 2025-07-16 23:00:00         đến 2025-08-02 11:00:00             interval: 4h


In [5]:
historical_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2025-07-16 23:00:00 to 2025-08-02 11:00:00
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   (Bitstamp:BTCUSD, open)    100 non-null    float64
 1   (Bitstamp:BTCUSD, high)    100 non-null    float64
 2   (Bitstamp:BTCUSD, low)     100 non-null    float64
 3   (Bitstamp:BTCUSD, close)   100 non-null    float64
 4   (Bitstamp:BTCUSD, volume)  100 non-null    float64
dtypes: float64(5)
memory usage: 4.7 KB


# **Indicators**

### SMA

In [35]:
def calculate_sma(prices: pd.Series, window: int) -> pd.Series:
    """
    Calculate Simple Moving Average
    
    Args:
        prices: Price series (typically closing prices)
        window: Window size for SMA calculation
        
    Returns:
        SMA series
    """
    return prices.rolling(window=window).mean()

def calculate_sma_crossover(prices: pd.Series, short_window: int, long_window: int) -> Tuple[pd.Series, pd.Series]:
    """
    Calculate SMA crossover components
    
    Args:
        prices: Price series (typically closing prices)
        short_window: Short SMA window
        long_window: Long SMA window
        
    Returns:
        Tuple of (short_sma, long_sma)
    """
    short_sma = calculate_sma(prices, short_window)
    long_sma = calculate_sma(prices, long_window)
    
    return short_sma, long_sma

def calculate_sma_components(prices: pd.Series, short_window: int, long_window: int) -> Dict[str, pd.Series]:
    """
    Calculate SMA crossover components and return as dictionary
    
    Args:
        prices: Price series (typically closing prices)
        short_window: Short SMA window
        long_window: Long SMA window
        
    Returns:
        Dictionary containing SMA components
    """
    short_sma, long_sma = calculate_sma_crossover(prices, short_window, long_window)
    
    return {
        'short_sma': short_sma,
        'long_sma': long_sma,
        'sma_diff': short_sma - long_sma
    }

### MACD

In [8]:
def calculate_macd(prices: pd.Series, fast_period: int = 12, 
                  slow_period: int = 26, signal_period: int = 9) -> Tuple[pd.Series, pd.Series, pd.Series]:
    """
    Calculate MACD indicator components
    
    Args:
        prices: Price series (typically closing prices)
        fast_period: Period for fast EMA (default: 12)
        slow_period: Period for slow EMA (default: 26)
        signal_period: Period for signal line EMA (default: 9)
        
    Returns:
        Tuple of (macd_line, signal_line, histogram)
    """
    # Calculate exponential moving averages
    ema_fast = prices.ewm(span=fast_period).mean()
    ema_slow = prices.ewm(span=slow_period).mean()
    
    # MACD line
    macd_line = ema_fast - ema_slow
    
    # Signal line
    signal_line = macd_line.ewm(span=signal_period).mean()
    
    # Histogram
    histogram = macd_line - signal_line
    
    return macd_line, signal_line, histogram


def calculate_macd_components(prices: pd.Series, fast_period: int = 12, 
                            slow_period: int = 26, signal_period: int = 9) -> dict:
    """
    Calculate all MACD components and return as dictionary
    
    Args:
        prices: Price series (typically closing prices)
        fast_period: Period for fast EMA (default: 12)
        slow_period: Period for slow EMA (default: 26)
        signal_period: Period for signal line EMA (default: 9)
        
    Returns:
        Dictionary containing all MACD components
    """
    macd_line, signal_line, histogram = calculate_macd(prices, fast_period, slow_period, signal_period)
    
    return {
        'macd_line': macd_line,
        'signal_line': signal_line,
        'histogram': histogram,
        'zero_line': pd.Series([0] * len(prices), index=prices.index)
    } 

### RSI

In [9]:
def calculate_rsi(prices: pd.Series, period: int = 14) -> pd.Series:
    """
    Calculate RSI (Relative Strength Index) indicator
    
    Args:
        prices: Price series (typically closing prices)
        period: Period for RSI calculation (default: 14)
        
    Returns:
        RSI series
    """
    delta = prices.diff()
    
    # Separate gains and losses
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    
    # Calculate average gains and losses
    avg_gains = gains.rolling(window=period).mean()
    avg_losses = losses.rolling(window=period).mean()
    
    # Calculate RS and RSI
    rs = avg_gains / avg_losses
    rsi = 100 - (100 / (1 + rs))
    
    return rsi


def calculate_rsi_components(prices: pd.Series, period: int = 14, 
                           overbought_threshold: int = 70, 
                           oversold_threshold: int = 30) -> Dict[str, pd.Series]:
    """
    Calculate RSI and related components
    
    Args:
        prices: Price series (typically closing prices)
        period: Period for RSI calculation (default: 14)
        overbought_threshold: Overbought level (default: 70)
        oversold_threshold: Oversold level (default: 30)
        
    Returns:
        Dictionary containing RSI and threshold lines
    """
    rsi = calculate_rsi(prices, period)
    
    return {
        'rsi': rsi,
        'overbought_line': pd.Series([overbought_threshold] * len(prices), index=prices.index),
        'oversold_line': pd.Series([oversold_threshold] * len(prices), index=prices.index)
    } 

# **Strategies**

### Base strategy

In [11]:
def update_position(symbol: str, quantity: float, price: float):
    """Update strategy's position tracking"""
    if symbol not in positions:
        positions[symbol] = {
            'quantity': 0,
            'avg_price': 0,
            'total_cost': 0
        }
    
    pos = positions[symbol]
    
    if quantity > 0:  # Buy
        total_quantity = pos['quantity'] + quantity
        total_cost = pos['total_cost'] + (quantity * price)
        pos['quantity'] = total_quantity
        pos['total_cost'] = total_cost
        pos['avg_price'] = total_cost / total_quantity
    else:  # Sell
        pos['quantity'] += quantity  # quantity is negative for sell
        if pos['quantity'] <= 0:
            # Position closed
            pos['quantity'] = 0
            pos['avg_price'] = 0
            pos['total_cost'] = 0
    
def get_position(symbol: str) -> Optional[Dict[str, Any]]:
    """Get current position for a symbol"""
    return positions.get(symbol)

def has_position(symbol: str) -> bool:
    """Check if strategy has a position in symbol"""
    return symbol in positions and positions[symbol]['quantity'] > 0

def calculate_pnl(symbol: str, current_price: float) -> float:
    """Calculate unrealized P&L for a position"""
    if not has_position(symbol):
        return 0.0
    
    position = positions[symbol]
    return (current_price - position['avg_price']) * position['quantity']

def reset():
    """Reset strategy state"""
    positions = {}
    signals = {}
    logger.info(f"Reset strategy: {name}")

def get_summary() -> Dict[str, Any]:
    """Get strategy summary"""
    return {
        'name': name,
        'positions': positions,
        'total_positions': len([p for p in positions.values() if p['quantity'] > 0])
    }

In [12]:
df.head()

,symbol,open,high,low,close,volume
datetime,,,,,,
2023-01-01 07:00:00,Bitstamp:BTCUSD,16530.0,16546.0,16507.0,16522.0,59.436695
2023-01-01 11:00:00,Bitstamp:BTCUSD,16523.0,16535.0,16496.0,16517.0,64.831896
2023-01-01 15:00:00,Bitstamp:BTCUSD,16517.0,16544.0,16498.0,16544.0,74.120038
2023-01-01 19:00:00,Bitstamp:BTCUSD,16544.0,16560.0,16535.0,16546.0,67.301095
2023-01-01 23:00:00,Bitstamp:BTCUSD,16546.0,16612.0,16546.0,16597.0,99.620520


In [13]:
symbol = 'Bitstamp:BTCUSD'
smacrossover = SMACrossoverStrategy(short_window=20, long_window=50)

# Try to call and print the output to debug
result = calculate_sma_components(df.close, smacrossover.short_window, smacrossover.long_window)

# If result is a tuple/list, get the first two elements
short_sma, long_sma = result['short_sma'], result['long_sma']
print(f"Short SMA (first 5):\n{short_sma.tail()}")
print(f"Long SMA (first 5):\n{long_sma.tail()}")


TypeError: SMACrossoverStrategy.__init__() got an unexpected keyword argument 'short_window'

### SMA Crossover

In [ ]:
class SMACrossoverStrategy(BaseStrategy):
    """Simple Moving Average Crossover Strategy"""
    
    def __init__(self, short_window: int, long_window: int):
        """Initialize SMA Crossover strategy"""
        # super().__init__("sma_crossover")
        
        # Strategy parameters
        self.short_window = short_window
        self.long_window = long_window
        
        # Validate parameters
        if self.short_window >= self.long_window:
            raise ValueError("Short window must be less than long window")
        
        print(f"SMA Crossover Strategy initialized: {self.short_window}/{self.long_window}")
        # logger.info(f"SMA Crossover Strategy initialized: {self.short_window}/{self.long_window}")
    
    def generate_signals(self, historical_data: pd.DataFrame, current_data: pd.Series) -> Dict[str, str]:
        """
        Generate trading signals based on SMA crossover
        
        Args:
            historical_data: Historical market data up to current point (OHLCV or close-only)
            current_data: Current day's market data
            
        Returns:
            Dict mapping symbol to signal ('buy', 'sell', 'hold')
        """
        signals = {}
        
        # Legacy close-only data structure
        for symbol in historical_data.symbol.unique():
            # if isinstance(symbol, str) and not symbol.endswith('_SMA'):
                # Get historical price data for this symbol
            symbol_data = historical_data[df.symbol==symbol]['close'].dropna()
            # print(symbol_data, symbol)
            
            if len(symbol_data) >= self.long_window:
                signal = self._generate_signal_for_symbol(symbol_data, symbol)
                signals[symbol] = signal
            else:
                signals[symbol] = 'hold'  # Not enough data
        
        return signals
    
    def _generate_signal_for_symbol(self, price_data: pd.Series, symbol: str) -> str:
        """Generate signal for a single symbol"""
        if len(price_data) < self.long_window:
            return 'hold'  # Not enough data
        
        # Calculate SMAs
        short_sma, long_sma = calculate_sma_crossover(price_data, self.short_window, self.long_window)

        # Get current and previous values
        current_short = short_sma.iloc[-1]
        current_long = long_sma.iloc[-1]
        prev_short = short_sma.iloc[-2] if len(short_sma) > 1 else current_short
        prev_long = long_sma.iloc[-2] if len(long_sma) > 1 else current_long
        
        # Check for crossover
        current_cross_up = current_short > current_long
        prev_cross_up = prev_short > prev_long
        
        # Generate signals
        if current_cross_up and not prev_cross_up:
            # Golden cross (short SMA crosses above long SMA)
            # logger.info(f"Golden cross detected for {symbol}: {current_short:.2f} > {current_long:.2f}")
            print(f"Golden cross detected for {symbol}: {current_short:.2f} > {current_long:.2f}")
            return 'buy'
        elif not current_cross_up and prev_cross_up:
            # Death cross (short SMA crosses below long SMA)
            # logger.info(f"Death cross detected for {symbol}: {current_short:.2f} < {current_long:.2f}")
            print(f"Death cross detected for {symbol}: {current_short:.2f} < {current_long:.2f}")
            return 'sell'
        else:
            return 'hold'
    
    def validate_config(self) -> bool:
        """Validate strategy configuration"""
        if self.short_window <= 0 or self.long_window <= 0:
            logger.error("SMA windows must be positive")
            return False
        
        if self.short_window >= self.long_window:
            logger.error("Short window must be less than long window")
            return False
        
        return True
    
    def get_indicators(self, price_data: pd.Series) -> Dict[str, pd.Series]:
        """Get strategy indicators for the given price data"""
        if len(price_data) < self.long_window:
            return {}
        
        # Calculate SMA components
        result = calculate_sma_components(price_data, self.short_window, self.long_window)
                
        return {
            f'{self.short_window}_SMA': result['short_sma'],
            f'{self.long_window}_SMA': result['long_sma'],
            f'diff': result['sma_diff']
        }
    
    def get_summary(self) -> Dict[str, Any]:
        """Get strategy summary"""
        return {
            'name': self.name,
            'strategy_type': 'SMA Crossover',
            'short_window': self.short_window,
            'long_window': self.long_window,
            'description': f'SMA Crossover with {self.short_window}/{self.long_window} periods'
        } 

### RSI

### Initalize strategies

In [9]:
# Cell 4: Thêm strategies
# SMA Crossover Strategy
sma_strategy = SMACrossoverStrategy({
    "short_window": engine.config.get("strategies.sma_crossover.short_window", 5),
    "long_window": engine.config.get("strategies.sma_crossover.long_window", 30),
    "name": "SMA_Crossover"
})
engine.add_strategy(sma_strategy)

# RSI Strategy
rsi_strategy = RSIStrategy({
    "period": engine.config.get("strategies.rsi.period", 14),
    "oversold": engine.config.get("strategies.rsi.oversold", 30),
    "overbought": engine.config.get("strategies.rsi.overbought", 70),
    "name": "RSI_Strategy"
})
engine.add_strategy(rsi_strategy)

# MACD Strategy
macd_strategy = MACDStrategy({
    "fast_period": engine.config.get("strategies.macd.fast_period", 15),
    "slow_period": engine.config.get("strategies.macd.slow_period", 20),
    "signal_period": engine.config.get("strategies.macd.signal_period", 7),
    "name": "MACD_Strategy"
})
engine.add_strategy(macd_strategy)

print(f"Đã thêm {len(engine.strategies)} strategies:")
for name, strategy in engine.strategies.items():
    print(f"  - {name}: {strategy.__class__.__name__}")

2025-08-02 14:57:22.516 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: sma_crossover
2025-08-02 14:57:22.518 | INFO     | src.strategies.sma_crossover:__init__:48 - SMA Crossover Strategy initialized: 5/30
2025-08-02 14:57:22.519 | INFO     | src.core.trading_engine:add_strategy:82 - Added strategy: sma_crossover
2025-08-02 14:57:22.521 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: rsi
2025-08-02 14:57:22.523 | INFO     | src.strategies.rsi_strategy:__init__:56 - RSI Strategy initialized: period=20, overbought=70, oversold=30
2025-08-02 14:57:22.525 | INFO     | src.core.trading_engine:add_strategy:82 - Added strategy: rsi
2025-08-02 14:57:22.526 | INFO     | src.strategies.base_strategy:__init__:34 - Initialized strategy: macd
2025-08-02 14:57:22.527 | INFO     | src.strategies.macd_strategy:__init__:54 - MACD Strategy initialized: fast=15, slow=20, signal=7
2025-08-02 14:57:22.528 | INFO     | src.core.trading_engine:add_

Đã thêm 3 strategies:
  - sma_crossover: SMACrossoverStrategy
  - rsi: RSIStrategy
  - macd: MACDStrategy


# **Optimizing**

In [ ]:
class ParameterGrid:
    """Defines parameter grids for different strategy types"""
    
    def __init__(self):
        """Initialize parameter grids for different strategies"""
        self.grids = self._initialize_grids()
    
    def _initialize_grids(self) -> Dict[str, Dict[str, List]]:
        """Initialize parameter grids for each strategy type"""
        return {
            'sma_crossover': {
                'short_window': [5, 10, 15, 20, 25, 30],
                'long_window': [30, 40, 50, 60, 70, 80, 90, 100]
            },
            'rsi': {
                'period': [10, 14, 20, 30],
                'oversold': [20, 25, 30, 35],
                'overbought': [65, 70, 75, 80]
            },
            'macd': {
                'fast_period': [8, 10, 12, 15, 20],
                'slow_period': [20, 26, 30, 35, 40],
                'signal_period': [7, 9, 12, 15]
            },
            'bollinger_bands': {
                'period': [10, 15, 20, 30],
                'std_dev': [1.5, 2.0, 2.5, 3.0]
            }
        }
    
    def get_parameter_combinations(self, strategy_type: str) -> List[Dict[str, Any]]:
        """
        Get all parameter combinations for a strategy type
        
        Args:
            strategy_type: Type of strategy ('sma_crossover', 'rsi', 'macd', etc.)
            
        Returns:
            List of parameter dictionaries
        """
        if strategy_type not in self.grids:
            logger.warning(f"No parameter grid defined for strategy type: {strategy_type}")
            return []
        
        grid = self.grids[strategy_type]
        param_names = list(grid.keys())
        param_values = list(grid.values())
        
        # Generate all combinations
        combinations = list(itertools.product(*param_values))
        
        # Convert to list of dictionaries
        param_combinations = []
        for combo in combinations:
            param_dict = dict(zip(param_names, combo))
            param_combinations.append(param_dict)
        
        logger.info(f"Generated {len(param_combinations)} parameter combinations for {strategy_type}")
        return param_combinations
    
    def get_filtered_combinations(self, strategy_type: str, 
                                filters: Dict[str, Any] = None) -> List[Dict[str, Any]]:
        """
        Get parameter combinations with optional filters
        
        Args:
            strategy_type: Type of strategy
            filters: Dictionary of filters to apply (e.g., {'short_window': [10, 20]})
            
        Returns:
            Filtered list of parameter dictionaries
        """
        combinations = self.get_parameter_combinations(strategy_type)
        
        if not filters:
            return combinations
        
        filtered_combinations = []
        for combo in combinations:
            include = True
            for param, allowed_values in filters.items():
                if param in combo and combo[param] not in allowed_values:
                    include = False
                    break
            if include:
                filtered_combinations.append(combo)
        
        logger.info(f"Filtered to {len(filtered_combinations)} combinations")
        return filtered_combinations
    
    def add_custom_grid(self, strategy_type: str, parameters: Dict[str, List]):
        """
        Add custom parameter grid for a strategy type
        
        Args:
            strategy_type: Name of the strategy type
            parameters: Dictionary of parameter names to value lists
        """
        self.grids[strategy_type] = parameters
        logger.info(f"Added custom parameter grid for {strategy_type}")
    
    def get_grid_info(self, strategy_type: str) -> Dict[str, Any]:
        """
        Get information about parameter grid for a strategy type
        
        Args:
            strategy_type: Type of strategy
            
        Returns:
            Dictionary with grid information
        """
        if strategy_type not in self.grids:
            return {}
        
        grid = self.grids[strategy_type]
        total_combinations = 1
        for values in grid.values():
            total_combinations *= len(values)
        
        return {
            'parameters': list(grid.keys()),
            'parameter_ranges': grid,
            'total_combinations': total_combinations
        }
    
    def validate_parameters(self, strategy_type: str, parameters: Dict[str, Any]) -> bool:
        """
        Validate if parameters are within the defined grid
        
        Args:
            strategy_type: Type of strategy
            parameters: Parameters to validate
            
        Returns:
            True if parameters are valid
        """
        if strategy_type not in self.grids:
            return False
        
        grid = self.grids[strategy_type]
        
        for param, value in parameters.items():
            if param in grid and value not in grid[param]:
                logger.warning(f"Parameter {param}={value} not in grid for {strategy_type}")
                return False
        
        return True
    
    def get_parameter_bounds(self, strategy_type: str) -> Dict[str, Tuple]:
        """
        Get parameter bounds for a strategy type
        
        Args:
            strategy_type: Type of strategy
            
        Returns:
            Dictionary of parameter bounds (min, max)
        """
        if strategy_type not in self.grids:
            return {}
        
        grid = self.grids[strategy_type]
        bounds = {}
        
        for param, values in grid.items():
            if values:
                bounds[param] = (min(values), max(values))
        
        return bounds 

In [ ]:
class StrategyOptimizer:
    """Optimizes strategy parameters using risk management metrics"""
    
    def __init__(self, config: ConfigManager):
        """Initialize strategy optimizer"""
        self.config = config
        self.backtest_engine = BacktestEngine(config)
        self.parameter_grid = ParameterGrid()
        
        # Strategy factory
        self.strategy_factory = {
            'sma_crossover': SMACrossoverStrategy,
            'rsi': RSIStrategy,
            'macd': MACDStrategy
        }
        
        # Optimization results
        self.optimization_results = {}
        self.best_parameters = {}
        
        logger.info("Strategy optimizer initialized")
    
    def optimize_strategy(self, strategy_type: str, 
                         start_date: str, end_date: str,
                         optimization_metric: str = 'sharpe_ratio',
                         max_combinations: int = None,
                         filters: Dict[str, Any] = None,
                         use_parallel: bool = True) -> Dict[str, Any]:
        """
        Optimize parameters for a specific strategy type
        
        Args:
            strategy_type: Type of strategy to optimize
            start_date: Start date for optimization period
            end_date: End date for optimization period
            optimization_metric: Metric to optimize ('sharpe_ratio', 'total_return', 'profit_factor', etc.)
            max_combinations: Maximum number of parameter combinations to test
            filters: Optional filters for parameter combinations
            use_parallel: Whether to use parallel processing
            
        Returns:
            Dictionary containing optimization results
        """
        logger.info(f"Starting optimization for {strategy_type}")
        
        # Get parameter combinations
        combinations = self.parameter_grid.get_filtered_combinations(strategy_type, filters)
        
        if max_combinations and len(combinations) > max_combinations:
            # Sample combinations if too many
            combinations = np.random.choice(combinations, max_combinations, replace=False).tolist()
            logger.info(f"Sampled {max_combinations} combinations from {len(combinations)} total")
        
        if not combinations:
            logger.error(f"No parameter combinations available for {strategy_type}")
            return {}
        
        # Run optimization
        if use_parallel and len(combinations) > 10:
            results = self._optimize_parallel(strategy_type, combinations, start_date, end_date, optimization_metric)
        else:
            results = self._optimize_sequential(strategy_type, combinations, start_date, end_date, optimization_metric)
        
        # Find best parameters
        best_params = self._find_best_parameters(results, optimization_metric)
        
        # Store results
        self.optimization_results[strategy_type] = {
            'results': results,
            'best_parameters': best_params,
            'optimization_metric': optimization_metric,
            'total_combinations': len(combinations),
            'tested_combinations': len(results)
        }
        
        self.best_parameters[strategy_type] = best_params
        
        logger.info(f"Optimization completed for {strategy_type}")
        logger.info(f"Best parameters: {best_params}")
        
        return self.optimization_results[strategy_type]
    
    def _optimize_sequential(self, strategy_type: str, combinations: List[Dict[str, Any]],
                           start_date: str, end_date: str, optimization_metric: str) -> List[Dict[str, Any]]:
        """Run optimization sequentially"""
        results = []
        
        for i, params in enumerate(combinations):
            try:
                result = self._test_parameter_combination(
                    strategy_type, params, start_date, end_date
                )
                result['parameters'] = params
                result['combination_index'] = i
                results.append(result)
                
                if (i + 1) % 10 == 0:
                    logger.info(f"Tested {i + 1}/{len(combinations)} combinations")
                    
            except Exception as e:
                logger.warning(f"Failed to test combination {i}: {e}")
                continue
        
        return results
    
    def _optimize_parallel(self, strategy_type: str, combinations: List[Dict[str, Any]],
                          start_date: str, end_date: str, optimization_metric: str) -> List[Dict[str, Any]]:
        """Run optimization using parallel processing"""
        results = []
        
        with ProcessPoolExecutor(max_workers=min(4, len(combinations))) as executor:
            # Submit all tasks
            future_to_params = {
                executor.submit(self._test_parameter_combination, strategy_type, params, start_date, end_date): params
                for params in combinations
            }
            
            # Collect results
            for i, future in enumerate(as_completed(future_to_params)):
                params = future_to_params[future]
                try:
                    result = future.result()
                    result['parameters'] = params
                    result['combination_index'] = i
                    results.append(result)
                    
                    if (i + 1) % 10 == 0:
                        logger.info(f"Completed {i + 1}/{len(combinations)} combinations")
                        
                except Exception as e:
                    logger.warning(f"Failed to test combination {i}: {e}")
                    continue
        
        return results
    
    def _test_parameter_combination(self, strategy_type: str, params: Dict[str, Any],
                                  start_date: str, end_date: str) -> Dict[str, Any]:
        """Test a single parameter combination"""
        try:
            # Create strategy with parameters
            strategy_class = self.strategy_factory.get(strategy_type)
            if not strategy_class:
                raise ValueError(f"Unknown strategy type: {strategy_type}")
            
            strategy = strategy_class(params)
            
            # Run backtest
            strategies = {f"{strategy_type}_test": strategy}
            results = self.backtest_engine.run_backtest(strategies, start_date, end_date)
            
            # Extract metrics
            strategy_result = results.get(f"{strategy_type}_test", {})
            performance_metrics = strategy_result.get('performance_metrics', {})
            risk_metrics = strategy_result.get('risk_metrics', {})
            
            return {
                'total_return': performance_metrics.get('total_return', 0),
                'annualized_return': performance_metrics.get('annualized_return', 0),
                'sharpe_ratio': performance_metrics.get('sharpe_ratio', 0),
                'max_drawdown': performance_metrics.get('max_drawdown', 0),
                'volatility': performance_metrics.get('volatility', 0),
                'win_rate': performance_metrics.get('win_rate', 0),
                'profit_factor': performance_metrics.get('profit_factor', 0),
                'total_trades': performance_metrics.get('total_trades', 0),
                'final_portfolio_value': performance_metrics.get('final_portfolio_value', 0),
                'current_drawdown': risk_metrics.get('current_drawdown', 0),
                'portfolio_volatility': risk_metrics.get('volatility', 0)
            }
            
        except Exception as e:
            logger.error(f"Error testing parameters {params}: {e}")
            return {
                'total_return': 0,
                'annualized_return': 0,
                'sharpe_ratio': -999,
                'max_drawdown': 0,
                'volatility': 0,
                'win_rate': 0,
                'profit_factor': 0,
                'total_trades': 0,
                'final_portfolio_value': 0,
                'current_drawdown': 0,
                'portfolio_volatility': 0
            }
    
    def _find_best_parameters(self, results: List[Dict[str, Any]], 
                             optimization_metric: str) -> Dict[str, Any]:
        """Find the best parameters based on optimization metric"""
        if not results:
            return {}
        
        # Sort by optimization metric
        valid_results = [r for r in results if r.get(optimization_metric) is not None]
        
        if not valid_results:
            return {}
        
        # Sort by optimization metric (descending for most metrics, ascending for drawdown)
        reverse = optimization_metric not in ['max_drawdown', 'volatility', 'current_drawdown']
        sorted_results = sorted(valid_results, 
                              key=lambda x: x.get(optimization_metric, 0), 
                              reverse=reverse)
        
        best_result = sorted_results[0]
        return {
            'parameters': best_result.get('parameters', {}),
            'metrics': {k: v for k, v in best_result.items() if k != 'parameters' and k != 'combination_index'},
            'rank': 1
        }
    
    def optimize_multiple_strategies(self, strategy_types: List[str],
                                   start_date: str, end_date: str,
                                   optimization_metric: str = 'sharpe_ratio',
                                   max_combinations_per_strategy: int = None) -> Dict[str, Any]:
        """
        Optimize multiple strategies
        
        Args:
            strategy_types: List of strategy types to optimize
            start_date: Start date for optimization period
            end_date: End date for optimization period
            optimization_metric: Metric to optimize
            max_combinations_per_strategy: Maximum combinations per strategy
            
        Returns:
            Dictionary containing results for all strategies
        """
        all_results = {}
        
        for strategy_type in strategy_types:
            logger.info(f"Optimizing {strategy_type}")
            
            try:
                result = self.optimize_strategy(
                    strategy_type, start_date, end_date, 
                    optimization_metric, max_combinations_per_strategy
                )
                all_results[strategy_type] = result
                
            except Exception as e:
                logger.error(f"Failed to optimize {strategy_type}: {e}")
                continue
        
        return all_results
    
    def get_optimization_summary(self, strategy_type: str = None) -> Dict[str, Any]:
        """Get summary of optimization results"""
        if strategy_type:
            if strategy_type not in self.optimization_results:
                return {}
            return self.optimization_results[strategy_type]
        
        return {
            'strategies': list(self.optimization_results.keys()),
            'best_parameters': self.best_parameters,
            'total_optimizations': len(self.optimization_results)
        }
    
    def get_top_parameters(self, strategy_type: str, top_n: int = 5, 
                          metric: str = 'sharpe_ratio') -> List[Dict[str, Any]]:
        """Get top N parameter combinations for a strategy"""
        if strategy_type not in self.optimization_results:
            return []
        
        results = self.optimization_results[strategy_type]['results']
        valid_results = [r for r in results if r.get(metric) is not None]
        
        if not valid_results:
            return []
        
        # Sort by metric
        reverse = metric not in ['max_drawdown', 'volatility', 'current_drawdown']
        sorted_results = sorted(valid_results, 
                              key=lambda x: x.get(metric, 0), 
                              reverse=reverse)
        
        top_results = []
        for i, result in enumerate(sorted_results[:top_n]):
            top_results.append({
                'rank': i + 1,
                'parameters': result.get('parameters', {}),
                'metrics': {k: v for k, v in result.items() 
                           if k not in ['parameters', 'combination_index']}
            })
        
        return top_results
    
    def save_optimization_results(self, filepath: str):
        """Save optimization results to file"""
        # Convert results to serializable format
        serializable_results = {}
        for strategy_type, result in self.optimization_results.items():
            serializable_results[strategy_type] = {
                'best_parameters': result['best_parameters'],
                'optimization_metric': result['optimization_metric'],
                'total_combinations': result['total_combinations'],
                'tested_combinations': result['tested_combinations'],
                'top_results': self.get_top_parameters(strategy_type, top_n=10)
            }
        
        with open(filepath, 'w') as f:
            json.dump(serializable_results, f, indent=2, default=str)
        
        logger.info(f"Optimization results saved to {filepath}")
    
    def load_optimization_results(self, filepath: str):
        """Load optimization results from file"""
        if not os.path.exists(filepath):
            logger.warning(f"Optimization results file not found: {filepath}")
            return
        
        with open(filepath, 'r') as f:
            data = json.load(f)
        
        # Convert back to internal format
        for strategy_type, result in data.items():
            self.best_parameters[strategy_type] = result.get('best_parameters', {})
        
        logger.info(f"Optimization results loaded from {filepath}")
    
    def create_optimized_strategy(self, strategy_type: str, 
                                use_best_parameters: bool = True,
                                custom_parameters: Dict[str, Any] = None) -> Any:
        """
        Create a strategy with optimized parameters
        
        Args:
            strategy_type: Type of strategy to create
            use_best_parameters: Whether to use best parameters from optimization
            custom_parameters: Custom parameters to use instead
            
        Returns:
            Strategy instance
        """
        if strategy_type not in self.strategy_factory:
            raise ValueError(f"Unknown strategy type: {strategy_type}")
        
        if use_best_parameters and strategy_type in self.best_parameters:
            params = self.best_parameters[strategy_type].get('parameters', {})
        elif custom_parameters:
            params = custom_parameters
        else:
            # Use default parameters
            params = {}
        
        strategy_class = self.strategy_factory[strategy_type]
        return strategy_class(params)
    
    def generate_optimization_report(self, strategy_type: str = None) -> str:
        """Generate a comprehensive optimization report"""
        if strategy_type and strategy_type not in self.optimization_results:
            return f"No optimization results available for {strategy_type}"
        
        report = []
        report.append("=" * 60)
        report.append("STRATEGY OPTIMIZATION REPORT")
        report.append("=" * 60)
        
        if strategy_type:
            self._add_strategy_report(report, strategy_type)
        else:
            for strategy_type in self.optimization_results.keys():
                self._add_strategy_report(report, strategy_type)
                report.append("-" * 40)
        
        return "\n".join(report)
    
    def _add_strategy_report(self, report: List[str], strategy_type: str):
        """Add strategy-specific report section"""
        result = self.optimization_results[strategy_type]
        best_params = result['best_parameters']
        
        report.append(f"\nStrategy: {strategy_type.upper()}")
        report.append(f"Optimization Metric: {result['optimization_metric']}")
        report.append(f"Total Combinations Tested: {result['tested_combinations']}")
        report.append(f"Best Parameters: {best_params.get('parameters', {})}")
        
        metrics = best_params.get('metrics', {})
        if metrics:
            report.append(f"Best Performance Metrics:")
            report.append(f"  Total Return: {metrics.get('total_return', 0):.2%}")
            report.append(f"  Annualized Return: {metrics.get('annualized_return', 0):.2%}")
            report.append(f"  Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.2f}")
            report.append(f"  Max Drawdown: {metrics.get('max_drawdown', 0):.2%}")
            report.append(f"  Win Rate: {metrics.get('win_rate', 0):.2%}")
            report.append(f"  Profit Factor: {metrics.get('profit_factor', 0):.2f}")
            report.append(f"  Total Trades: {metrics.get('total_trades', 0)}")
    
    def plot_optimization_results(self, strategy_type: str, save_path: str = None):
        """Plot optimization results for a strategy"""
        if strategy_type not in self.optimization_results:
            logger.warning(f"No optimization results for {strategy_type}")
            return
        
        import matplotlib.pyplot as plt
        
        results = self.optimization_results[strategy_type]['results']
        
        # Create scatter plot of parameter combinations vs performance
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle(f'Optimization Results: {strategy_type.upper()}', fontsize=16)
        
        # Extract data for plotting
        sharpe_ratios = [r.get('sharpe_ratio', 0) for r in results]
        total_returns = [r.get('total_return', 0) for r in results]
        max_drawdowns = [r.get('max_drawdown', 0) for r in results]
        win_rates = [r.get('win_rate', 0) for r in results]
        
        # Sharpe ratio distribution
        axes[0, 0].hist(sharpe_ratios, bins=20, alpha=0.7)
        axes[0, 0].set_title('Sharpe Ratio Distribution')
        axes[0, 0].set_xlabel('Sharpe Ratio')
        axes[0, 0].grid(True)
        
        # Total return vs max drawdown
        axes[0, 1].scatter(max_drawdowns, total_returns, alpha=0.6)
        axes[0, 1].set_title('Return vs Drawdown')
        axes[0, 1].set_xlabel('Max Drawdown')
        axes[0, 1].set_ylabel('Total Return')
        axes[0, 1].grid(True)
        
        # Win rate vs profit factor
        axes[1, 0].scatter(win_rates, [r.get('profit_factor', 0) for r in results], alpha=0.6)
        axes[1, 0].set_title('Win Rate vs Profit Factor')
        axes[1, 0].set_xlabel('Win Rate')
        axes[1, 0].set_ylabel('Profit Factor')
        axes[1, 0].grid(True)
        
        # Performance metrics comparison
        metrics = ['sharpe_ratio', 'total_return', 'win_rate', 'profit_factor']
        metric_values = []
        for metric in metrics:
            values = [r.get(metric, 0) for r in results]
            metric_values.append(np.mean(values))
        
        axes[1, 1].bar(metrics, metric_values)
        axes[1, 1].set_title('Average Performance Metrics')
        axes[1, 1].set_ylabel('Average Value')
        axes[1, 1].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        else:
            plt.show()
        
        plt.close() 

# **Backtest**

In [10]:
# Cell 5: Chạy backtest
print("Bắt đầu chạy backtest...")

# Chạy chiến lược
print('SMA Crossover:', engine.config.get('strategies.sma_crossover.enabled', False))
print('RSI:', engine.config.get('strategies.rsi.enabled', False))
print('MACD:', engine.config.get('strategies.macd.enabled', False), '\n')

# Chạy backtest với chiến lược đã chọn
engine.run_backtest(start_date, end_date)

2025-08-02 14:57:30.276 | INFO     | src.core.trading_engine:run_backtest:86 - Starting backtest from 2023-01-01 to 2025-07-31
2025-08-02 14:57:30.277 | INFO     | src.data.data_manager:get_historical_data_standardized:589 - Fetching standardized OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2025-07-31
2025-08-02 14:57:30.278 | INFO     | src.data.data_manager:get_historical_data:64 - Fetching historical OHLCV data for ['Bitstamp:BTCUSD'] from 2023-01-01 to 2025-07-31


Bắt đầu chạy backtest...
SMA Crossover: True
RSI: False
MACD: False 



2025-08-02 14:57:30.669 | INFO     | src.data.data_manager:_fetch_tradingview_ohlcv_data:159 - Successfully fetched OHLCV data for 1 symbols
2025-08-02 14:57:30.671 | INFO     | src.data.data_manager:get_historical_data:97 - OHLCV data cached successfully
2025-08-02 14:57:30.675 | INFO     | src.data.data_manager:ensure_ohlcv_format:500 - Data formatted for Bitstamp:BTCUSD: (100, 5)
2025-08-02 14:57:30.676 | INFO     | src.data.data_manager:get_historical_data_standardized:622 - Successfully standardized OHLCV data for 1 symbols
2025-08-02 14:57:30.680 | INFO     | src.core.trading_engine:run_backtest:107 - Loaded 100 data points for [('Bitstamp:BTCUSD', 'open'), ('Bitstamp:BTCUSD', 'high'), ('Bitstamp:BTCUSD', 'low'), ('Bitstamp:BTCUSD', 'close'), ('Bitstamp:BTCUSD', 'volume')]
2025-08-02 14:57:30.697 | INFO     | src.strategies.sma_crossover:_generate_signal_for_symbol:109 - Golden cross detected for Bitstamp:BTCUSD: 118563.60 > 118355.17
2025-08-02 14:57:30.699 | INFO     | src.core

# **Showing Results**

In [11]:
# Cell 8: Hiển thị chi tiết trades
if hasattr(engine, 'trades') and engine.trades:
    print(f"\n=== CHI TIẾT TRADES ({len(engine.trades)} trades) ===")
    
    trades_df = pd.DataFrame([
        {
            'Symbol': trade.symbol,
            'Side': trade.side,
            'Quantity': trade.quantity,
            'Price': trade.price,
            'Timestamp': trade.timestamp,
            'Commission': trade.commission,
            'Strategy': trade.strategy
        }
        for trade in engine.trades
    ])
    
    print(trades_df.head(10))
    
    # Trade statistics
    print(f"\n=== TRADE STATISTICS ===")
    print(f"Total Trades: {len(engine.trades)}")
    print(f"Buy Trades: {len(trades_df[trades_df['Side'] == 'buy'])}")
    print(f"Sell Trades: {len(trades_df[trades_df['Side'] == 'sell'])}")
    print(f"Average Trade Size: {trades_df['Quantity'].mean():.2f}")
    print(f"Total Commission: ${trades_df['Commission'].sum():.2f}")
else:
    print("Không có trades nào được thực hiện")


=== CHI TIẾT TRADES (6 trades) ===
            Symbol  Side  Quantity     Price           Timestamp  Commission  \
0  Bitstamp:BTCUSD   buy  0.167583  119344.0 2025-07-22 23:00:00   20.000000   
1  Bitstamp:BTCUSD  sell  0.167583  115400.0 2025-07-25 07:00:00   19.339053   
2  Bitstamp:BTCUSD   buy  0.167959  118243.0 2025-07-27 07:00:00   19.859943   
3  Bitstamp:BTCUSD  sell  0.167959  118144.0 2025-07-30 07:00:00   19.843315   
4  Bitstamp:BTCUSD   buy  0.167797  118290.0 2025-07-31 19:00:00   19.848677   
5  Bitstamp:BTCUSD  sell  0.167797  116767.0 2025-07-31 23:00:00   19.593122   

        Strategy  
0  sma_crossover  
1  sma_crossover  
2  sma_crossover  
3  sma_crossover  
4  sma_crossover  
5  sma_crossover  

=== TRADE STATISTICS ===
Total Trades: 6
Buy Trades: 3
Sell Trades: 3
Average Trade Size: 0.17
Total Commission: $118.48


In [12]:
# Cell 6: Lấy kết quả backtest
portfolio_summary = engine.get_portfolio_summary()

print("=== KẾT QUẢ BACKTEST ===")
print(f"Initial Capital: ${portfolio_summary['initial_capital']:,.2f}")
print(f"Final Portfolio Value: ${portfolio_summary['total_value']:,.2f}")
print(f"Total Return: {portfolio_summary['total_return']:.2%}")
print(f"Annualized Return: {portfolio_summary['annualized_return']:.2%}")
print(f"Sharpe Ratio: {portfolio_summary['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {portfolio_summary['max_drawdown']:.2%}")
print(f"Win Rate: {portfolio_summary['win_rate']:.2%}")
print(f"Total Trades: {portfolio_summary['total_trades']}")

# Cell 7: Visualize portfolio performance
def plot_portfolio_performance(engine):
    """Plot portfolio performance"""
    portfolio_history = engine.portfolio_history
    
    if not portfolio_history:
        print("Không có dữ liệu portfolio history")
        return
    
    df = pd.DataFrame(portfolio_history)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    # Create subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=('Portfolio Value', 'Daily Returns', 'Cumulative Returns', 
                       'Drawdown', 'Cash vs Positions', 'Trade Distribution'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Portfolio Value
    fig.add_trace(
        go.Scatter(x=df.index, y=df['total_value'], 
                  mode='lines', name='Portfolio Value'),
        row=1, col=1
    )
    
    # Daily Returns
    daily_returns = df['total_value'].pct_change()
    fig.add_trace(
        go.Scatter(x=df.index, y=daily_returns, 
                  mode='lines', name='Daily Returns'),
        row=1, col=2
    )
    
    # Cumulative Returns
    cumulative_returns = (1 + daily_returns).cumprod()
    fig.add_trace(
        go.Scatter(x=df.index, y=cumulative_returns, 
                  mode='lines', name='Cumulative Returns'),
        row=2, col=1
    )
    
    # Drawdown
    running_max = df['total_value'].expanding().max()
    drawdown = (df['total_value'] - running_max) / running_max
    fig.add_trace(
        go.Scatter(x=df.index, y=drawdown, 
                  mode='lines', name='Drawdown', fill='tonexty'),
        row=2, col=2
    )
    
    # Cash vs Positions
    fig.add_trace(
        go.Scatter(x=df.index, y=df['cash'], 
                  mode='lines', name='Cash'),
        row=3, col=1
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df['total_value'] - df['cash'], 
                  mode='lines', name='Positions'),
        row=3, col=1
    )
    
    # Trade Distribution (if available)
    if hasattr(engine, 'trades') and engine.trades:
        trade_returns = [trade.pnl for trade in engine.trades]
        fig.add_trace(
            go.Histogram(x=trade_returns, name='Trade Returns'),
            row=3, col=2
        )
    
    fig.update_layout(height=900, title_text="Portfolio Performance Analysis")
    fig.show()

# Chạy visualization
plot_portfolio_performance(engine)

=== KẾT QUẢ BACKTEST ===
Initial Capital: $100,000.00
Final Portfolio Value: $98,948.39
Total Return: -1.05%
Annualized Return: -2.66%
Sharpe Ratio: -2.31
Max Drawdown: -1.14%
Win Rate: 0.00%
Total Trades: 6


In [13]:
def viz(symbol, trades_df, historical_data):
    # Visualize
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.8, 0.2])
    fig.add_trace(go.Candlestick(x=historical_data[symbol].index,
                    open=historical_data[symbol].open,
                    high=historical_data[symbol].high,
                    low=historical_data[symbol].low,
                    close=historical_data[symbol].close,
                    ), row=1, col=1)
    set_ylim = (historical_data[symbol].low.min() * 0.98, historical_data[symbol].high.max() * 1.02)
    # Add buy/sell markers from trades_df to the candlestick chart (row 1)
    if 'trades_df' in locals():
        buy_trades = trades_df[trades_df['Side'] == 'buy']
        sell_trades = trades_df[trades_df['Side'] == 'sell']
        # Buy markers
        fig.add_trace(
            go.Scatter(
                x=buy_trades['Timestamp'],
                y=historical_data[symbol][historical_data.index.isin(buy_trades['Timestamp'])].low * 0.998,
                mode='markers+text',
                marker=dict(symbol='triangle-up', color='green', size=12),
                text=['Buy']*len(buy_trades),
                textposition='bottom center',
                name='Buy'
            ),
            row=1, col=1
        )
        # Sell markers
        fig.add_trace(
            go.Scatter(
                x=sell_trades['Timestamp'],
                y=historical_data[symbol][historical_data.index.isin(sell_trades['Timestamp'])].high * 1.002,
                mode='markers+text',
                marker=dict(symbol='triangle-down', color='red', size=12),
                text=['Sell']*len(sell_trades),
                textposition='top center',
                name='Sell'
            ),
            row=1, col=1
        )

    fig.add_trace(go.Bar(x=historical_data[symbol].index,
                        y=historical_data[symbol].volume,
                        ), row=2, col=1)

    fig.update_layout(title=f'{symbol}',
                    yaxis_range=(set_ylim[0], set_ylim[1]),
                    xaxis_title='Date',
                    yaxis_title='Price', 
                    height=800, width=1000)
    fig.show()

viz(symbols[0], trades_df, historical_data)